# VacationPy

In [1]:
# -- dependencies and setup ---
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# --- store the filepath in a variable ---
file = "../output_data/cities.csv"

# --- read the csv file and create a dataframe ---
cities_data = pd.DataFrame(pd.read_csv(file, index_col="City ID"))

# --- display the dataframe ---
cities_data

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City ID,,,,,,,,,
0,Broome,US,2020-06-19 23:23:58,42.25,-75.83,77.00,73,90,5.32
1,Saint Paul Harbor,US,2020-06-19 23:23:31,57.79,-152.41,53.60,71,1,14.99
2,Rikitea,PF,2020-06-19 23:25:15,-23.12,-134.97,74.61,82,100,20.94
3,Qaanaaq,GL,2020-06-19 23:25:15,77.48,-69.36,36.54,79,91,1.01
4,Quatre Cocos,MU,2020-06-19 23:25:15,-20.21,57.76,71.60,68,20,16.11
...,...,...,...,...,...,...,...,...,...
563,Vestmannaeyjar,IS,2020-06-19 23:26:18,63.44,-20.27,50.92,83,100,15.05
564,Antalaha,MG,2020-06-19 23:26:18,-14.90,50.28,70.52,96,100,11.45
565,Ambon City,ID,2020-06-19 23:26:18,-3.70,128.18,77.00,94,40,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# --- access maps with unique API key ---
gmaps.configure(api_key=g_key)

# --- use Lat and Lng as locations and Humidity as the weight ---
locations = cities_data[["Lat", "Lng"]]
weights = cities_data["Humidity (%)"]

# --- customize the size of the figure and add heatmap layer to map ---
figure_layout = {
    'width': "100%",
    'height': '400px',
#    'border': '1px solid black',
    'padding': '1px',
#    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
fig.add_layer(heatmap_layer)

# --- set maximum intensity and point_radius settings ---
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

# --- display heatmap ---
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# --- create dataframe with conditional statements to fit the below criteria ---
# 1. A max temperature lower than 80 degrees but higher than 70.
# 2. Wind speed less than 10 mph.
# 3. Zero cloudiness.
# 4. Humidity is less than 75%
# 5

hotel_df = cities_data.loc[(cities_data["Max Temp (F)"] > 70) & 
                               (cities_data["Max Temp (F)"] < 80) & 
                               (cities_data["Wind Speed (mph)"] < 10) & 
                               (cities_data["Cloudiness (%)"] == 0) &
                               (cities_data["Humidity (%)"] < 75), :].reset_index(drop=True).dropna()
hotel_df

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Simplício Mendes,BR,2020-06-19 23:25:21,-7.85,-41.91,75.70,60,0,4.99
1,Andradina,BR,2020-06-19 23:25:30,-20.90,-51.38,70.84,62,0,6.13
2,Avarua,CK,2020-06-19 23:22:31,-21.21,-159.78,75.20,57,0,9.17
3,Abnūb,EG,2020-06-19 23:25:54,27.27,31.15,78.80,39,0,9.17
4,Presidencia Roque Sáenz Peña,AR,2020-06-19 23:26:03,-26.79,-60.44,79.00,66,0,3.80
5,Roviés,GR,2020-06-19 23:26:04,38.82,23.23,75.00,42,0,5.01
6,Marrakesh,MA,2020-06-19 23:26:08,31.63,-8.01,73.40,49,0,6.93
7,Kununurra,AU,2020-06-19 23:26:08,-15.77,128.73,73.40,38,0,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# --- Add a "Hotel Name" column to the DataFrame ---
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Simplício Mendes,BR,2020-06-19 23:25:21,-7.85,-41.91,75.70,60,0,4.99,
1,Andradina,BR,2020-06-19 23:25:30,-20.90,-51.38,70.84,62,0,6.13,
2,Avarua,CK,2020-06-19 23:22:31,-21.21,-159.78,75.20,57,0,9.17,
3,Abnūb,EG,2020-06-19 23:25:54,27.27,31.15,78.80,39,0,9.17,
4,Presidencia Roque Sáenz Peña,AR,2020-06-19 23:26:03,-26.79,-60.44,79.00,66,0,3.80,
5,Roviés,GR,2020-06-19 23:26:04,38.82,23.23,75.00,42,0,5.01,
6,Marrakesh,MA,2020-06-19 23:26:08,31.63,-8.01,73.40,49,0,6.93,
7,Kununurra,AU,2020-06-19 23:26:08,-15.77,128.73,73.40,38,0,5.82,


In [6]:
# --- set up parameters for querying Google Places API ---
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# --- set up a parameters dictionary ---
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# --- define base url ---
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [7]:
# --- use a for loop to go through each city in the dataframe and make an API call ---

for index, row in hotel_df.iterrows():
    
    # --- get the coordinates and store in params dictionary ---
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # --- call the API and get hotel details for each city ---
    hotel_data = requests.get(base_url, params).json()
    
    # --- use exception handling to save the first hotel name for each city ---
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City {row['City']} found! The nearest hotel is {hotel_name}")
        print("-----------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-----------------------------------------------------------------")
        

City Simplício Mendes found! The nearest hotel is Hotel Fidalgo
-----------------------------------------------------------------
City Andradina found! The nearest hotel is Savana Park Hotel
-----------------------------------------------------------------
City Avarua found! The nearest hotel is The Edgewater Resort and Spa
-----------------------------------------------------------------
Could not find information for city 'Abnūb'... skipping.
-----------------------------------------------------------------
City Presidencia Roque Sáenz Peña found! The nearest hotel is Hotel Apart
-----------------------------------------------------------------
City Roviés found! The nearest hotel is hotel Agnanti
-----------------------------------------------------------------
City Marrakesh found! The nearest hotel is Riad Kasbah
-----------------------------------------------------------------
City Kununurra found! The nearest hotel is Hotel Kununurra
---------------------------------------------

In [8]:
# --- replace any missing values with np.nan ---
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)

# --- display new hotel dataframe ---
hotel_df


,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Simplício Mendes,BR,2020-06-19 23:25:21,-7.85,-41.91,75.70,60,0,4.99,Hotel Fidalgo
1,Andradina,BR,2020-06-19 23:25:30,-20.90,-51.38,70.84,62,0,6.13,Savana Park Hotel
2,Avarua,CK,2020-06-19 23:22:31,-21.21,-159.78,75.20,57,0,9.17,The Edgewater Resort and Spa
3,Abnūb,EG,2020-06-19 23:25:54,27.27,31.15,78.80,39,0,9.17,NaN
4,Presidencia Roque Sáenz Peña,AR,2020-06-19 23:26:03,-26.79,-60.44,79.00,66,0,3.80,Hotel Apart
5,Roviés,GR,2020-06-19 23:26:04,38.82,23.23,75.00,42,0,5.01,hotel Agnanti
6,Marrakesh,MA,2020-06-19 23:26:08,31.63,-8.01,73.40,49,0,6.93,Riad Kasbah
7,Kununurra,AU,2020-06-19 23:26:08,-15.77,128.73,73.40,38,0,5.82,Hotel Kununurra


In [9]:
# --- remove null rows from dataframe ---
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# --- display final dataframe ---
hotel_df

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Simplício Mendes,BR,2020-06-19 23:25:21,-7.85,-41.91,75.70,60,0,4.99,Hotel Fidalgo
1,Andradina,BR,2020-06-19 23:25:30,-20.90,-51.38,70.84,62,0,6.13,Savana Park Hotel
2,Avarua,CK,2020-06-19 23:22:31,-21.21,-159.78,75.20,57,0,9.17,The Edgewater Resort and Spa
3,Presidencia Roque Sáenz Peña,AR,2020-06-19 23:26:03,-26.79,-60.44,79.00,66,0,3.80,Hotel Apart
4,Roviés,GR,2020-06-19 23:26:04,38.82,23.23,75.00,42,0,5.01,hotel Agnanti
5,Marrakesh,MA,2020-06-19 23:26:08,31.63,-8.01,73.40,49,0,6.93,Riad Kasbah
6,Kununurra,AU,2020-06-19 23:26:08,-15.77,128.73,73.40,38,0,5.82,Hotel Kununurra


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# --- add marker layer on top of heat map ---
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))